In [1]:
import ftplib, re
from pathlib import Path
import pandas as pd, numpy as np
from urllib.request import urlretrieve

# Download files

## List files

In [ ]:
FTP = "ftp.ncbi.nlm.nih.gov"
DIR = "/geo/series/GSE49nnn/GSE49828/suppl/"
PAT = re.compile(r"RRBS_.*\.bed\.txt\.gz$")

with ftplib.FTP(FTP) as ftp:
    ftp.login()                 # anonymous
    ftp.cwd(DIR)
    rrbs_files = [f for f in ftp.nlst() if PAT.search(f)]

print(len(rrbs_files), "RRBS files on server")


32 RRBS files on server


In [9]:
class GSE(object):
    def __init__(self, gse_id):
        self.gse_id = gse_id
        self.download_link = f"ftp.ncbi.nlm.nih.gov/geo/series/{gse_id[:5]}nnn/{gse_id}/suppl/"
        self.FTP = "ftp.ncbi.nlm.nih.gov"
        self.DIR = f"geo/series/{gse_id[:5]}nnn/{gse_id}/suppl/"
        self.dir_raw = f"data/raw/{gse_id}"
        self.dir_collapsed = f"data/collapsed/{gse_id}"
        self.path_list = f"data/{gse_id}_rrbs_files.txt"
    def __str__(self):
        return f"GSE object for {self.gse_id}"
    def setup(self):
        if Path(f"{self.path_list}").exists():
            print("RRBS files list already exists, skipping setup.")
            return
        if Path(f"{self.dir_raw}").exists():
            print("Raw GSE data directory already exists.")
            return
        else:
            Path(f"{self.dir_raw}").mkdir(parents=True, exist_ok=True)
            print(f"Created directory for raw GSE data: {self.dir_raw}")
        if Path(f"{self.dir_collapsed}").exists():
            print("Collapsed GSE data directory already exists.")
            return
        else:
            Path(f"{self.dir_collapsed}").mkdir(parents=True, exist_ok=True)
            print(f"Created directory for collapsed GSE data: {self.dir_collapsed}")
        PAT = re.compile(r"RRBS_.*\.bed\.txt\.gz$")
        with ftplib.FTP(self.FTP) as ftp:
            ftp.login()                 # anonymous
            ftp.cwd(self.DIR)
            rrbs_files = [f for f in ftp.nlst() if PAT.search(f)]
        with open(f"{self.path_list}", "w") as f:
            for file in rrbs_files:
                f.write(file + "\n")
        print(f"Created list of RRBS files: {self.path_list}")
        print(len(rrbs_files), "RRBS files on server")    
    def download(self): 
        with open(self.path_list, "r") as f:
            rrbs_files = [line.strip() for line in f if line.strip()]     
        for fname in rrbs_files:
            sample_id = fname.replace(".txt.gz", "")
            feather_path = Path(self.dir_collapsed) / f"{sample_id}.feather"
            gz_path = Path(self.dir_raw) / fname
            if feather_path.exists():
                print(f"✓ Already processed: {sample_id}")
                continue
            if not gz_path.exists():
                print(f"↓ Downloading {fname}...")
                try:
                    urlretrieve(Path(self.download_link) / fname, gz_path)
                except Exception as e:
                    print(f"⚠️ Failed to download {fname}: {e}")
                    continue
            else:
                print(f"✓ Already downloaded: {fname}")
gse = GSE("GSE51239")
print(gse.download_link)
print(gse)
gse.setup()


ftp.ncbi.nlm.nih.gov/geo/series/GSE51nnn/GSE51239/suppl/
GSE object for GSE51239
Created directory for raw GSE data: data/raw/GSE51239
Created directory for collapsed GSE data: data/collapsed/GSE51239
Created list of RRBS files: data/GSE51239_rrbs_files.txt
0 RRBS files on server


# Download listed files

In [3]:
raw_dir      = Path("data/raw")
feather_dir  = Path("data/collapsed")
raw_dir.mkdir(exist_ok=True, parents=True)
feather_dir.mkdir(exist_ok=True, parents=True)

In [4]:
def collapse_one(path, cov_min=5):
    DTYPES = dict(chr="category", pos="int32", ref="category", chain="category",
                  total="int16", met="int16", unmet="int16",
                  rate="float32", ctx="category", type="category")
    COLS = ["chr","pos","ref","chain","total","met","unmet","rate","ctx","type"]

    out_chunks = []
    for chunk in pd.read_table(path, sep=r"\s+", names=COLS, dtype=DTYPES,
                               chunksize=1_000_000, header=0):
        chunk = chunk.query("type == 'CpG' and total >= @cov_min").copy()
        chunk["cpg_pos"] = np.where(chunk.chain == "-", chunk.pos - 1, chunk.pos)
        grp = (chunk.groupby(["chr","cpg_pos"], observed=True, as_index=False)
                    .agg(met=("met", "sum"), total=("total", "sum")))
        out_chunks.append(grp)

    df = pd.concat(out_chunks, ignore_index=True)
    df["rate"] = df.met / df.total
    return df[["chr","cpg_pos","rate"]]

In [ ]:

BASE = "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE49nnn/GSE49828/suppl/"

for fname in rrbs_files:
    sample_id = fname.replace("GSE49828_RRBS_", "").replace("_methylation_calling.bed.txt.gz", "")
    feather_path = feather_dir / f"{sample_id}.feather"
    gz_path = raw_dir / fname

    if feather_path.exists():
        print(f"✓ Already processed: {sample_id}")
        continue

    if not gz_path.exists():
        print(f"↓ Downloading {fname}...")
        try:
            urlretrieve(BASE + fname, gz_path)
        except Exception as e:
            print(f"⚠️ Failed to download {fname}: {e}")
            continue
    else:
        print(f"✓ Already downloaded: {fname}")

    print(f"⚙️  Processing: {sample_id}")
    try:
        df = collapse_one(gz_path)
        df.rename(columns={"rate": sample_id}).to_feather(feather_path)
        print(f"✔ Done: {sample_id}")
        del df
    except Exception as e:
        print(f"⚠️ Failed to process {sample_id}: {e}")


✓ Already downloaded: GSE49828_RRBS_1st_PB1_methylation_calling.bed.txt.gz
⚙️  Processing: 1st_PB1
✔ Done: 1st_PB1
↓ Downloading GSE49828_RRBS_1st_PB2_methylation_calling.bed.txt.gz...
⚙️  Processing: 1st_PB2
✔ Done: 1st_PB2
↓ Downloading GSE49828_RRBS_1st_PB3_methylation_calling.bed.txt.gz...
⚙️  Processing: 1st_PB3
✔ Done: 1st_PB3
✓ Already downloaded: GSE49828_RRBS_2-cell1_methylation_calling.bed.txt.gz
⚙️  Processing: 2-cell1
✔ Done: 2-cell1
↓ Downloading GSE49828_RRBS_2-cell2_methylation_calling.bed.txt.gz...
⚙️  Processing: 2-cell2
✔ Done: 2-cell2
↓ Downloading GSE49828_RRBS_2nd_PB1_methylation_calling.bed.txt.gz...
⚙️  Processing: 2nd_PB1
✔ Done: 2nd_PB1
↓ Downloading GSE49828_RRBS_2nd_PB2_methylation_calling.bed.txt.gz...
⚙️  Processing: 2nd_PB2
✔ Done: 2nd_PB2
↓ Downloading GSE49828_RRBS_4-cell1_methylation_calling.bed.txt.gz...
⚙️  Processing: 4-cell1
✔ Done: 4-cell1
↓ Downloading GSE49828_RRBS_4-cell2_methylation_calling.bed.txt.gz...
⚙️  Processing: 4-cell2
✔ Done: 4-cell2


In [11]:
import os
x=os.listdir(feather_dir)  # List files in the feather directory
len(x)

33

# Concatenate files

In [12]:
from functools import reduce

feathers = sorted(feather_dir.glob("*.feather"))
dfs = []

for f in feathers:
    name = f.stem
    df = pd.read_feather(f)[["chr","cpg_pos", name]]
    dfs.append(df)

matrix = reduce(lambda l,r: pd.merge(l,r,on=["chr","cpg_pos"], how="outer"), dfs)
matrix.sort_values(["chr","cpg_pos"], inplace=True)


In [16]:
# remove column named "1st" from matrix
matrix=matrix.loc[:, matrix.columns != "1st"]
matrix

,chr,cpg_pos,1st_PB1,1st_PB2,1st_PB3,2-cell1,2-cell2,2nd_PB1,2nd_PB2,4-cell1,...,Postimplantation_embryo3,Sperm1,Sperm2,Sperm3,Sperm4,TE1,TE2,TE3,Zygote1,Zygote2
0,chr1,10609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,10617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,10620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,10631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,10867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4374954,chrY,59357712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,1.000000,0.6,1.000000,NaN,NaN,0.333333,NaN,NaN
4374955,chrY,59357736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,1.000000,1.0,0.666667,NaN,NaN,0.666667,NaN,NaN
4374956,chrY,59357765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4374957,chrY,59357786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.8,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# save the final matrix
matrix.to_parquet("data/GSE49828_methylation_matrix.parquet")

# Check out result

In [4]:
df = pd.read_parquet("data/GSE49828_methylation_matrix.parquet")
df

,chr,cpg_pos,1st_PB1,1st_PB2,1st_PB3,2-cell1,2-cell2,2nd_PB1,2nd_PB2,4-cell1,...,Postimplantation_embryo3,Sperm1,Sperm2,Sperm3,Sperm4,TE1,TE2,TE3,Zygote1,Zygote2
0,chr1,10609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,10617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,10620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,10631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,10867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4374954,chrY,59357712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,1.000000,0.6,1.000000,NaN,NaN,0.333333,NaN,NaN
4374955,chrY,59357736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,1.000000,1.0,0.666667,NaN,NaN,0.666667,NaN,NaN
4374956,chrY,59357765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4374957,chrY,59357786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.8,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['chr', 'cpg_pos', '1st_PB1', '1st_PB2', '1st_PB3', '2-cell1', '2-cell2',
       '2nd_PB1', '2nd_PB2', '4-cell1', '4-cell2', '8-cell1', '8-cell2',
       '8-cell3', 'ICM1', 'ICM2', 'ICM3', 'MII_Oocyte1', 'MII_Oocyte2',
       'Morula1', 'Morula2', 'Morula3', 'Postimplantation_embryo1',
       'Postimplantation_embryo2', 'Postimplantation_embryo3', 'Sperm1',
       'Sperm2', 'Sperm3', 'Sperm4', 'TE1', 'TE2', 'TE3', 'Zygote1',
       'Zygote2'],
      dtype='object')

# Liftover to hg38 from hg19

In [112]:
liftover_df = pd.read_csv("data/GSE49828_cpgs_hg38.bed", sep="\t", header=None, names=["new_chrom", "start", "end", "site_name"])
liftover_df

,new_chrom,start,end,site_name
0,chr1,10608,10609,chr1_10609
1,chr1,10616,10617,chr1_10617
2,chr1,10619,10620,chr1_10620
3,chr1,10630,10631,chr1_10631
4,chr1,10866,10867,chr1_10867
...,...,...,...,...
4374141,chrY,57211560,57211561,chrY_59357712
4374142,chrY,57211584,57211585,chrY_59357736
4374143,chrY,57211613,57211614,chrY_59357765
4374144,chrY,57211634,57211635,chrY_59357786


In [117]:
liftover_df[["old_chrom", "old_pos"]] = liftover_df["site_name"].str.split("_", expand=True)
liftover_df["old_pos"] = liftover_df["old_pos"].astype(int)
liftover_df


,new_chrom,start,end,site_name,old_chrom,old_pos
0,chr1,10608,10609,chr1_10609,chr1,10609
1,chr1,10616,10617,chr1_10617,chr1,10617
2,chr1,10619,10620,chr1_10620,chr1,10620
3,chr1,10630,10631,chr1_10631,chr1,10631
4,chr1,10866,10867,chr1_10867,chr1,10867
...,...,...,...,...,...,...
4374141,chrY,57211560,57211561,chrY_59357712,chrY,59357712
4374142,chrY,57211584,57211585,chrY_59357736,chrY,59357736
4374143,chrY,57211613,57211614,chrY_59357765,chrY,59357765
4374144,chrY,57211634,57211635,chrY_59357786,chrY,59357786


In [114]:
# check old and new chromosome names
liftover_df[liftover_df["old_chrom"] != liftover_df["new_chrom"]]["old_chrom"].value_counts()

old_chrom
chr1     724
chr2     189
chr15    175
chr22    138
chr19    129
chr7     101
chrX      82
chr17     75
chr14     40
chr9      28
chr11     16
chr10      4
chrY       4
chr8       2
chr20      1
Name: count, dtype: int64

# Check sequence of cpg positions

In [8]:
import pyranges as pr
import pyfaidx

In [9]:
! curl -l https://hgdownload.soe.ucsc.edu/goldenpath/hg38/bigZips/hg38.fa.gz > data/hg38.fa.gz
! gunzip data/hg38.fa.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  938M  100  938M    0     0  11.3M      0  0:01:22  0:01:22 --:--:-- 15.8M    0    0     0      0      0 --:--:--  0:00:03 --:--:--     0


In [ ]:
# index with samtools
! samtools faidx data/hg38.fa

In [118]:
embryo_cpgs = liftover_df.copy()
embryo_cpgs["cpg_pos"] = embryo_cpgs["end"].astype(int)
embryo_cpgs["Start"] = embryo_cpgs["cpg_pos"] - 1
embryo_cpgs["End"] = embryo_cpgs["cpg_pos"] +1
embryo_cpgs

,new_chrom,start,end,site_name,old_chrom,old_pos,cpg_pos,Start,End
0,chr1,10608,10609,chr1_10609,chr1,10609,10609,10608,10610
1,chr1,10616,10617,chr1_10617,chr1,10617,10617,10616,10618
2,chr1,10619,10620,chr1_10620,chr1,10620,10620,10619,10621
3,chr1,10630,10631,chr1_10631,chr1,10631,10631,10630,10632
4,chr1,10866,10867,chr1_10867,chr1,10867,10867,10866,10868
...,...,...,...,...,...,...,...,...,...
4374141,chrY,57211560,57211561,chrY_59357712,chrY,59357712,57211561,57211560,57211562
4374142,chrY,57211584,57211585,chrY_59357736,chrY,59357736,57211585,57211584,57211586
4374143,chrY,57211613,57211614,chrY_59357765,chrY,59357765,57211614,57211613,57211615
4374144,chrY,57211634,57211635,chrY_59357786,chrY,59357786,57211635,57211634,57211636


In [120]:
# value count where positions are not the same
embryo_cpgs[embryo_cpgs["old_pos"] != embryo_cpgs["cpg_pos"]]

,new_chrom,start,end,site_name,old_chrom,old_pos,cpg_pos,Start,End
211,chr1,267342,267343,chr1_237094,chr1,237094,267343,267342,267344
212,chr1,267544,267545,chr1_237296,chr1,237296,267545,267544,267546
213,chr1,267549,267550,chr1_237301,chr1,237301,267550,267549,267551
214,chr1,267579,267580,chr1_237331,chr1,237331,267580,267579,267581
215,chr1,267594,267595,chr1_237346,chr1,237346,267595,267594,267596
...,...,...,...,...,...,...,...,...,...
4374141,chrY,57211560,57211561,chrY_59357712,chrY,59357712,57211561,57211560,57211562
4374142,chrY,57211584,57211585,chrY_59357736,chrY,59357736,57211585,57211584,57211586
4374143,chrY,57211613,57211614,chrY_59357765,chrY,59357765,57211614,57211613,57211615
4374144,chrY,57211634,57211635,chrY_59357786,chrY,59357786,57211635,57211634,57211636


In [121]:
embryo_cpgs[embryo_cpgs["old_pos"] == embryo_cpgs["cpg_pos"]]

,new_chrom,start,end,site_name,old_chrom,old_pos,cpg_pos,Start,End
0,chr1,10608,10609,chr1_10609,chr1,10609,10609,10608,10610
1,chr1,10616,10617,chr1_10617,chr1,10617,10617,10616,10618
2,chr1,10619,10620,chr1_10620,chr1,10620,10620,10619,10621
3,chr1,10630,10631,chr1_10631,chr1,10631,10631,10630,10632
4,chr1,10866,10867,chr1_10867,chr1,10867,10867,10866,10868
...,...,...,...,...,...,...,...,...,...
4366138,chrY,19287,19288,chrY_19288,chrY,19288,19288,19287,19289
4366139,chrY,19297,19298,chrY_19298,chrY,19298,19298,19297,19299
4366140,chrY,19307,19308,chrY_19308,chrY,19308,19308,19307,19309
4366141,chrY,19324,19325,chrY_19325,chrY,19325,19325,19324,19326


In [122]:
x = embryo_cpgs[:1000].copy()
x

,new_chrom,start,end,site_name,old_chrom,old_pos,cpg_pos,Start,End
0,chr1,10608,10609,chr1_10609,chr1,10609,10609,10608,10610
1,chr1,10616,10617,chr1_10617,chr1,10617,10617,10616,10618
2,chr1,10619,10620,chr1_10620,chr1,10620,10620,10619,10621
3,chr1,10630,10631,chr1_10631,chr1,10631,10631,10630,10632
4,chr1,10866,10867,chr1_10867,chr1,10867,10867,10866,10868
...,...,...,...,...,...,...,...,...,...
995,chr1,905563,905564,chr1_840944,chr1,840944,905564,905563,905565
996,chr1,906023,906024,chr1_841404,chr1,841404,906024,906023,906025
997,chr1,906095,906096,chr1_841476,chr1,841476,906096,906095,906097
998,chr1,906183,906184,chr1_841564,chr1,841564,906184,906183,906185


In [126]:
import pysam
fasta = pysam.FastaFile("data/hg38.fa")
def fetch_seq(row):
    return fasta.fetch(row["new_chrom"], row["Start"], row["End"])

In [127]:
%time
embryo_cpgs["Seq"] = embryo_cpgs.apply(fetch_seq, axis=1)
embryo_cpgs

CPU times: user 8 μs, sys: 1e+03 ns, total: 9 μs
Wall time: 21.2 μs


,new_chrom,start,end,site_name,old_chrom,old_pos,cpg_pos,Start,End,Seq
0,chr1,10608,10609,chr1_10609,chr1,10609,10609,10608,10610,cg
1,chr1,10616,10617,chr1_10617,chr1,10617,10617,10616,10618,cg
2,chr1,10619,10620,chr1_10620,chr1,10620,10620,10619,10621,cg
3,chr1,10630,10631,chr1_10631,chr1,10631,10631,10630,10632,cg
4,chr1,10866,10867,chr1_10867,chr1,10867,10867,10866,10868,cg
...,...,...,...,...,...,...,...,...,...,...
4374141,chrY,57211560,57211561,chrY_59357712,chrY,59357712,57211561,57211560,57211562,CG
4374142,chrY,57211584,57211585,chrY_59357736,chrY,59357736,57211585,57211584,57211586,CG
4374143,chrY,57211613,57211614,chrY_59357765,chrY,59357765,57211614,57211613,57211615,CG
4374144,chrY,57211634,57211635,chrY_59357786,chrY,59357786,57211635,57211634,57211636,CG


In [128]:
embryo_cpgs["Seq"].value_counts()

Seq
CG    2297895
cg    2048242
Cg       6298
cG       6158
gg       3198
GG       2665
gc       2298
GC       2114
ga       1453
gt       1175
GA       1052
GT        854
ca        169
CA        150
tg        142
TG        106
CC         42
cc         29
CT         26
ag         15
ct         14
AG         14
ac          7
AT          5
aa          4
Gc          3
Gg          3
TC          2
at          2
gG          2
TT          2
AC          2
Gt          1
Ga          1
cA          1
ta          1
Tg          1
Name: count, dtype: int64

In [104]:
cpgs_df["Seq"].value_counts()

Seq
tt    181814
aa    179850
TT    172139
AA    170652
CC    156060
       ...  
gT       230
cG       197
Cg       184
nn         2
cn         1
Name: count, Length: 68, dtype: int64

In [131]:
embryo_cpgs[["new_chrom", "cpg_pos", "old_chrom", "old_pos"]].to_csv("data/embryo_cpgs_h19_to_h38.csv", index=False)